In [ ]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch_geometric.data import DataLoader
from torch_geometric.nn import GCNConv
from tqdm import tqdm
import numpy as np

In [26]:
## Reading in Data

play_data = pd.read_csv("data/plays.csv")
player_data = pd.read_csv("data/players.csv")
player_play_data = pd.read_csv("data/player_play.csv")

passing_plays = play_data[play_data["passResult"].isin(["C", "I", "IN"])]

In [27]:
## Helper Functions

# Calculate distance between nodes
def get_edge_weights(edges, play, tracking_data):
    # Initializing weights array
    weights = []

    for i in range(len(edges[0])):
        # Getting 2 players from tracking data
        P1 = tracking_data[(tracking_data["gameId"] == play.gameId) & (tracking_data["playId"] == play.playId) & (tracking_data["nflId"] == edges[0][i])]
        P2 = tracking_data[(tracking_data["gameId"] == play.gameId) & (tracking_data["playId"] == play.playId) & (tracking_data["nflId"] == edges[1][i])]

        # Converting players to points
        point1 = np.array([P1["x"], P1["y"]])
        point2 = np.array([P2["x"], P2["y"]])

        # Calculating distance between players
        dist = np.linalg.norm(point2 - point1)

        weights.append(dist)

    return weights

# Finding targeted node
def get_targeted_receiver(Recs, nflIDs):
    
    for WR in Recs.itertuples(index=True):
        if (WR.wasTargettedReceiver):
            return np.where(nflIDs == WR.nflId)[0][0]

In [31]:
def prepare_data(graph):
    # Extract the edges, edge_weights, receivers, and defenders
    edge_index = torch.tensor(graph['edges'], dtype=torch.long)
    edge_weights = torch.tensor(graph['edge_weights'], dtype=torch.float)
    
    # Concatenate receivers and defenders to create node features
    receivers = torch.tensor(graph['receivers'], dtype=torch.float)
    defenders = torch.tensor(graph['defenders'], dtype=torch.float)
    x = torch.cat((receivers, defenders), dim=0)  # Combine receivers and defenders as node features
    
    # Labels (target values)
    y = torch.tensor([graph['y']], dtype=torch.float).view(-1)  # Assuming 'y' is the target label
    
    # Create a PyTorch Geometric Data object
    data = Data(x=x, edge_index=edge_index, edge_attr=edge_weights, y=y)
    return data

In [ ]:
class GNNModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(GNNModel, self).__init__()
        self.conv1 = GCNConv(input_dim, hidden_dim)
        self.conv2 = GCNConv(hidden_dim, output_dim)
        self.fc = nn.Linear(output_dim, 1)  # Assuming regression task for openness prediction

    def forward(self, x, edge_index, edge_attr):
        x = self.conv1(x, edge_index, edge_attr)
        x = torch.relu(x)
        x = self.conv2(x, edge_index, edge_attr)
        x = torch.relu(x)
        x = self.fc(x)
        return x

In [28]:

# Getting tracking data at moment of pass
week_data = pd.read_csv("data/tracking_week_1.csv")
week_data = week_data[week_data["event"].isin(["pass_forward", "pass_shovel"])]

# Getting list of completions by gameId and playId
completions = passing_plays[passing_plays["passResult"] == "C"]
playIds = completions[["gameId", "playId"]].drop_duplicates()

# Filtering tracking data to only completion plays
week_data_completions = week_data.merge(playIds, on=["gameId", "playId"])

# Removing playIds not from this week
playIds = playIds[playIds["gameId"].isin(set(week_data_completions["gameId"]))]

graph_data = []

for i,row in tqdm(enumerate(playIds.itertuples(index=True)), total=len(playIds), desc="Processing Plays", unit="play"):

    # Initializing play_data
    play_data = {}

    # Getting list of eligible receivers and defenders in coverage
    Recs = player_play_data[(player_play_data["gameId"] == row.gameId) & (player_play_data["playId"] == row.playId) & (player_play_data["wasRunningRoute"] == True)]
    Defs = player_play_data[(player_play_data["gameId"] == row.gameId) & (player_play_data["playId"] == row.playId) & ~pd.isna(player_play_data["pff_defensiveCoverageAssignment"])]

    # Creating edges array:  size = [2, len(Receivers) * len(Defenders)]
    R_nodes = np.arange(len(Recs))
    D_nodes = np.arange(len(Recs), len(Defs) + len(Recs))
    play_data["edges"] = np.vstack([np.repeat(R_nodes, len(Defs)), np.tile(D_nodes, len(Recs))])

    # Creating edges array with nflIds
    R_nodes2 = Recs["nflId"].values
    D_nodes2 = Defs["nflId"].values
    nflIDs = np.concatenate([Recs["nflId"].values, Defs["nflId"].values])
    edgeIDs = np.vstack([np.repeat(R_nodes2, len(Defs)), np.tile(D_nodes2, len(Recs))])

    # Getting distance from each offensive player to each defensive player
    play_data["edge_weights"] = get_edge_weights(edgeIDs, row, week_data_completions)

    # Creating vectors of node data for each player
    # Current Vector = [ x, y, speed, acceleration, orientation, direction of motion ]
    Receivers = []
    for WR in Recs.itertuples(index=True):
        WR_row = week_data_completions[(week_data_completions["gameId"] == row.gameId) & (week_data_completions["playId"] == row.playId) & (week_data_completions["nflId"] == WR.nflId)]
        Receivers.append([WR_row["x"].iloc[0], WR_row["y"].iloc[0], WR_row["s"].iloc[0], WR_row["a"].iloc[0], WR_row["o"].iloc[0], WR_row["dir"].iloc[0]])
    Defenders = []
    for Def in Defs.itertuples(index=True):
        Def_row = week_data_completions[(week_data_completions["gameId"] == row.gameId) & (week_data_completions["playId"] == row.playId) & (week_data_completions["nflId"] == Def.nflId)]
        Defenders.append([Def_row["x"].iloc[0], Def_row["y"].iloc[0], Def_row["s"].iloc[0], Def_row["a"].iloc[0], Def_row["o"].iloc[0], Def_row["dir"].iloc[0]])

    # Adding Receiver and Defender arrays to play_data
    play_data["receivers"] = Receivers
    play_data["defenders"] = Defenders

    play_data["y"] = get_targeted_receiver(Recs, nflIDs)

    graph_data.append(play_data)

Processing Plays: 100%|██████████| 696/696 [00:32<00:00, 21.31play/s]


In [ ]:
print(graph_data)
data_list = []
for graph in graph_data:
    data = prepare_data(graph)
    data_list.append(data)

input_dim = data.x.size(1)  # Features per node (6 for both receivers and defenders)
hidden_dim = 64
output_dim = 32  # You can adjust this based on your task
model = GNNModel(input_dim, hidden_dim, output_dim)

# Print the model
print(model)

[{'edges': array([[ 0,  0,  0,  0,  0,  0,  0,  1,  1,  1,  1,  1,  1,  1,  2,  2,
         2,  2,  2,  2,  2,  3,  3,  3,  3,  3,  3,  3],
       [ 4,  5,  6,  7,  8,  9, 10,  4,  5,  6,  7,  8,  9, 10,  4,  5,
         6,  7,  8,  9, 10,  4,  5,  6,  7,  8,  9, 10]]), 'edge_weights': [5.818358875146836, 14.723926106850712, 14.563145264674112, 10.315643460298539, 12.812700730134926, 9.58124209066862, 16.588441759249115, 17.916143558254937, 1.5730543537971007, 22.735824154844266, 23.018032061842298, 17.158484781588385, 9.380644967165104, 6.552289676136116, 11.131253298708103, 20.470889086700655, 5.895125104694555, 13.084846197032657, 27.233013054012222, 9.718893969994731, 15.669285242154476, 15.970704430299874, 17.30891388851421, 12.996865006608326, 19.250958417699625, 28.362122628604514, 8.765300907555883, 10.577395709719857], 'receivers': [[45.78, 29.65, 8.77, 0.95, 359.29, 314.98], [47.45, 15.77, 7.54, 4.36, 358.26, 337.37], [60.54, 29.46, 5.8, 1.48, 167.7, 199.35], [61.71, 22.45, 5